## Initialize notebook

In [18]:
import sagemaker as sm

# Specify the ARN of the model package you will be using. You can get this
# from the sagemaker console after you subscribe to the model package.
model_package_arn = 'arn:aws:sagemaker:us-east-2:214666064132:model-package/trends-styles-important-basic'


# This notebook creates an endpoint for realtime inference. You may replace the folmaylowing with any name you like. 
endpoint_name = 'prosper-test-endpoint'


## Realtime Inference Example

### Deploy model to endpoint

In [19]:
# Create model from model package
model = sm.ModelPackage(
            role=sm.get_execution_role(),
            model_package_arn=model_package_arn,
            sagemaker_session=sm.Session())

# Deploy the model to an endpoint. Be sure to delete the endpoint when you are finished with it.
# By default, this method waits until the endpoint is deployed. This could take a while.
# To have the API return immediately, set the wait parameter to false.
model.deploy(1, 'ml.m4.xlarge', endpoint_name=endpoint_name)

--------------------------------------------------------------------------------------!

### Use endpoint created above to do realtime prediction

In [21]:
# create predictor for endpoint created above0
predictor = sm.predictor.RealTimePredictor(endpoint_name, content_type='text/csv')

# Sample request
# Female
gender = 0
# Age Range: 18-24
age_range = 1
# Household Income: $75,000 to $79,999
household_income_range = 14

# format request data as comma-delimited string
request_data = f'{gender},{age_range},{household_income_range}'

# Submit the request to the endpoint.
# By default, the result is returned as a sequence of bytes. We decode it as utf-8 string.
# Note that there are parameters available for serializing and deserializing input and output data.
result = predictor.predict('1,2,2').decode('utf-8')

# print the result. This is the probability that the person with the requested parameters are in the
# target class.
print(f'Probability: {result}')

Probability: 0.5500407218933105


### Delete the endpoint

In [22]:
sm.session.Session().delete_endpoint(endpoint_name)

## Process Whole File with Batch Transform Job

### Prepare input file

In [23]:
# A sample input file (data/basic_test_data.csv) is provided with this notebook.
# The transform job expects its input file to to live in S3.  We upload the file
# to the default bucket with key_prefix of prosper-sample-data/basic-test-data. You
# can upload to any bucket and key you like if you specify the bucket and key_prefix parameters.

transform_input = sm.Session().upload_data('data/basic_test_data.csv', key_prefix='prosper-sample-data/basic-test-data')
print(f'transform input: {transform_input}')

transform input: s3://sagemaker-us-east-2-214666064132/prosper-sample-data/basic-test-data/basic_test_data.csv


### Create the transform job

In [24]:
# Create model from model package
model = sm.ModelPackage(
            role=sm.get_execution_role(),
            model_package_arn=model_package_arn,
            sagemaker_session=sm.Session())

# Create the transformer
# A variety of parameters may be specified here including the output path where
# SageMaker will send the results of the transform. Since we do not specify the output,
# Sagemaker will leave the results in the default bucket. We will retrieve this location 
# below so that we can inspect the output.
transformer = model.transformer(1, 'ml.m4.xlarge')

# Run the transform job.
# By default, the output file contains only the inference result for each row.
# You can use the output_filter parameter to include any of the input columns. Review also
# input_filter which allows you to filter the parameters passed as input to the transformer.
# The combination of input_filter and output_filter gives you a lot of flexibility.
# By default, the API does not wait for the transform job to complete. You can control this with
# the wait parameter.
transformer.transform(transform_input, content_type='text/csv')

# The transform job sets the output path in the output_path member.
print(f'Transform output: {transformer.output_path}')

Transform output: s3://sagemaker-us-east-2-214666064132/trends-styles-important-basic-2019-11-2-2019-11-22-18-17-23-276
